In [1]:
from __future__ import print_function

First we load the functions and libraries necessary for this report.

In [2]:
run analysis_functions.ipynb #import all helper functions

In [3]:
# Import main dataset
df = pd.read_csv('lastfm_9000_users.csv', na_filter=False)
df = df.drop(['Unnamed: 0'], axis=1)

The structure of the analysis will be as such: 
1. Objective
2. Prepare data
3. Model Fitting, Tuning, and Evaluation
    - Benchmarks
        - Most Popular
        - ALS Matrix Factorization
    - LightFM
        - Vanilla FM / BPR
        - FM with User/Item Side Information
        - Parameter Tuning
4. Model Exploration
    - Metrics Used (NDCG, Recall, Precision, Coverage)
    - Performance of each Model (Table)
    - NDCG Metric by User Type 
        - Active/Non-Active (Aggregate Plays)
        - Diverse/Non-Diverse (Top 1000 Artists)
        - "Basic"/Non-"Basic" (Popular)
    - Scale:
        - NDCG by Size
        - Training Time / Predict Time
6. Conclusion / Next Steps

# 0. Data Exploration

Let's see how the plays are distributed. 

In [4]:
import seaborn as sns
import matplotlib.pyplot as plt

In [5]:
sns.distplot(df.plays)

/anaconda/lib/python3.6/site-packages/matplotlib/axes/_axes.py:6571: UserWarning: The 'normed' kwarg is deprecated, and has been replaced by the 'density' kwarg.
  warnings.warn("The 'normed' kwarg is deprecated, and has been "


In [6]:
sns.distplot(df[df.plays < 1000].plays)

/anaconda/lib/python3.6/site-packages/matplotlib/axes/_axes.py:6571: UserWarning: The 'normed' kwarg is deprecated, and has been replaced by the 'density' kwarg.
  warnings.warn("The 'normed' kwarg is deprecated, and has been "


In [7]:
from sklearn.preprocessing import MinMaxScaler 

df['log_plays'] = df.plays.apply(lambda x: log(x))

df.log_plays *= (1.0/df.log_plays.max())

In [8]:
sns.distplot(df.log_plays)

/anaconda/lib/python3.6/site-packages/matplotlib/axes/_axes.py:6571: UserWarning: The 'normed' kwarg is deprecated, and has been replaced by the 'density' kwarg.
  warnings.warn("The 'normed' kwarg is deprecated, and has been "


# 1. Objective

BLALBALBALBLA

# 2. Prepare Data

### 2a. Create Sparse Matrix from Dataset

In [9]:
#create sparse matrix
plays_sparse = create_sparse_matrix(df).astype('float')
print('Matrix Sparsity:', calculate_sparsity(plays_sparse))

Creating sparse matrix...


/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:23: FutureWarning: specifying 'categories' or 'ordered' in .astype() is deprecated; pass a CategoricalDtype instead
/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:24: FutureWarning: specifying 'categories' or 'ordered' in .astype() is deprecated; pass a CategoricalDtype instead


Matrix Sparsity: 99.8965986346416


### 2b. Split Data to Train/Test

Split data into train test set, maintaining that each user still has some interactions intact.

In [ ]:
# Split data into training and test sets
train, test, user_count = split_train_test_per_user(plays_sparse, 3, 10)
print("Percentage of original data masked:", pct_masked(plays_sparse, train.T.tocsr()))
print("Users masked:", user_count)

In [ ]:
#train is item by user to accomodate implicit and baseline training
train

#### Scaled Train/Test

In [ ]:
df_log = df.drop(columns = ['plays'], axis = 1)
# plays_sparse = create_sparse_matrix(df).astype('float')

In [ ]:
df_log.columns = ['user_id','artist_mbid','artist_name','plays']

In [ ]:
#create sparse matrix
plays_sparse_log = create_sparse_matrix(df_log).astype('float')
print('Matrix Sparsity:', calculate_sparsity(plays_sparse_log))

# Split data into training and test sets
train_log, test_log, user_count_log = split_train_test_per_user(plays_sparse_log, 3, 10)
print("Percentage of original data masked:", pct_masked(plays_sparse_log, train_log.T.tocsr()))
print("Users masked:", user_count_log)

# 3. Model Fitting, Tuning, and Evaluation

- Note on evaluation: use metrics
- what autotune does

## 3a. Benchmarks

### Baseline

Baseline recommends the most-popular artists to everyone.

In [ ]:
model_baseline = Baseline(n_recs = 20)
model_baseline.fit(train)

No tuning is necessary since there are no parameters. We then evaluate the test set below: 

In [ ]:
coverage, precision, recall, ndcg = evaluate(model_baseline, "baseline", test, plays_sparse)
print("Precision:",precision*100,'%')
print("Recall:",recall*100,'%')
print("Coverage:",coverage*100,'%')
print("Average NDCG per User:",ndcg*100,'%')

### Model-Based (ALS)

Here we fit the model-based ALS Matrix Factorization using the implicit package from Homework 2 and use the parameters that were found to be optimized in the HW 2 report. 

In [ ]:
model_als = implicit.als.AlternatingLeastSquares(factors = 30, regularization = 0.01)

# Train model
print("Fitting model...")
model_als.fit(train)

In [ ]:
coverage, precision, recall, ndcg = evaluate(model_als, "implicit", test, train.T.tocsr())
print("Precision:",precision*100,'%')
print("Recall:",recall*100,'%')
print("Coverage:",coverage*100,'%')
print("Average NDCG per User:",ndcg*100,'%')

### LightFM (without side information)

Here we fit the LightFM model.

In [ ]:
from lightfm import LightFM

model_fm_vanilla = LightFM(learning_rate = .05, loss='warp')

# Train Model
print("Fitting model...")
model_fm_vanilla.fit(train_log.T.tocsr(),user_features = None, item_features = None)


Now we try to find the best hyperparameter for this model and use the model with the best hyperparameter to get our results.

In [ ]:
#coverage, precision, recall, ndcg = evaluate(model_fm_vanilla, "lightfm", test, plays_sparse,liked=train.T.tocsr())
coverage, precision, recall, ndcg = evaluate(model_fm_vanilla, "lightfm", test_log, plays_sparse_log, liked = train_log)
print("Precision:",precision*100,'%')
print("Recall:",recall*100,'%')
print("Coverage:",coverage*100,'%')
print("Average NDCG per User:",ndcg*100,'%')

In [ ]:
# from lightfm.cross_validation import random_train_test_split
# from lightfm.evaluation import precision_at_k
# from lightfm.evaluation import recall_at_k

# train, test = random_train_test_split(plays_sparse_log)
# recall_at_k(model_fm_vanilla, test, k = 20).mean()

### LightFM (with side information)

In [ ]:
run artist_features.ipynb

In [ ]:
#insert side info here
user_features = None
item_features = None

model_fm_features = LightFM(no_components=30, loss='warp')

# Train Model
print("Fitting model...")
model_fm_features.fit(train.T.tocsr(),user_features, item_features)

Now we try to find the best hyperparameter for thi smodel with the best hyperparameter to get our results.

### Summary of Performance Results (Table)

# 4. Model Exploration

Next, we explore these models even more. How do they perform in regards to size, different user population?

## 4a. Performance by User Type

In [11]:
run grouping_functions.ipynb

### Active/Non-Active

In [13]:
import sns

In [ ]:
n=3
activity_groups=active_users(plays_sparse, n)
levels=[1,2,3]
ndcg_activity = []
recall_activity = []
coverage_activity = []
precision_activity = []
model = implicit.als.AlternatingLeastSquares(factors=30, regularization=0.01)
for group in activity_groups:
    print('Matrix Sparsity:', calculate_sparsity(group))
    # Compute the recall and ndcg using optimal parameters for the ALS model on different dataset sizes
    train, test, user_count = split_train_test_per_user(group, 4, 20)
    # train model 
    print("Fitting model...")
    
    model.fit(train, show_progress=True)
    
    coverage, precision, recall, ndcg = evaluate(model,"implicit", test, plays_sparse)
    print("Recall:",recall*100,'%')
    print("NDCG:",ndcg*100,'%')
    print("Coverage:",coverage*100,'%')
    print("Precision:",precision*100,'%')

    recall_activity.append(recall)
    ndcg_activity.append(ndcg)
    precision_activity.append(precision)
    coverage_activity.append(coverage)

In [ ]:
# Plot scalability of ALS model
ndcg_activity_df = pd.DataFrame({'N':levels, 'NDCG': ndcg_activity})
g = sns.pointplot(x='N', y='NDCG', data=ndcg_activity_df)
plt.title('NDCG by Activity Level for ALS')
plt.xlabel('Activity Level')
plt.show()

### Diverse/Non-Diverse

In [ ]:
n=3
diversity_groups=diverse_users(plays_sparse, n)
levels=[1,2,3]
ndcg_div = []
recall_div = []
model = implicit.als.AlternatingLeastSquares(factors=30, regularization=0.01)
for group in diversity_groups:
    print('Matrix Sparsity:', calculate_sparsity(group))
    # Compute the recall and ndcg using optimal parameters for the ALS model on different dataset sizes
    train, test, user_count = split_train_test_per_user(group, 4, 20)
    # train model 
    print("Fitting model...")
    
    model.fit(train, show_progress=True)
    
    recall, ndcg = evaluate(model, "implicit",test, plays_sparse)
    print("Recall:",recall*100,'%')
    print("Average NDCG:",ndcg*100,'%')
    recall_div.append(recall)
    ndcg_div.append(ndcg)

In [ ]:
# Plot scalability of ALS model
ndcg_div_df = pd.DataFrame({'N':levels, 'NDCG': ndcg_div})
g = sns.pointplot(x='N', y='NDCG', data=ndcg_div_df)
plt.title('NDCG by Diversity Level for ALS')
plt.xlabel('Diversity Level')
plt.show()

### Basic/Non-Basic

In [ ]:
n=3
mainstream_groups=mainstream_users(plays_sparse, n)
levels=[1,2,3]
ndcg_mainstream = []
recall_mainstream = []
model = implicit.als.AlternatingLeastSquares(factors=30, regularization=0.01)
for group in mainstream_groups:
    print('Matrix Sparsity:', calculate_sparsity(group))
    # Compute the recall and ndcg using optimal parameters for the ALS model on different dataset sizes
    train, test, user_count = split_train_test_per_user(group, 4, 20)
    # train model 
    print("Fitting model...")
    
    model.fit(train, show_progress=True)
    
    recall, ndcg = evaluate(model, test, plays_sparse)
    print("Recall:",recall*100,'%')
    print("Average NDCG:",ndcg*100,'%')
    recall_mainstream.append(recall)
    ndcg_mainstream.append(ndcg)

In [ ]:
# Plot scalability of ALS model
ndcg_mainstream_df = pd.DataFrame({'N':levels, 'NDCG': ndcg_mainstream})
g = sns.pointplot(x='N', y='NDCG', data=ndcg_mainstream_df)
plt.title('NDCG by Mainstream-ness Level for ALS')
plt.xlabel('Mainstream-ness Level')
plt.show()

## 4b. Performance by Input Size 

### Accuracy/NDCG

In [ ]:
size = [9000,20000,40000,60000,80000]
import time
train_time = list()
ndcg_size = []
recall_size = []
#get train data for each input size
for users in size:
    df = get_users(df_150, users)
    plays_sparse = create_sparse_matrix(df)

    # K-Nearest Neighbors
    model_fm_features = LightFM(no_components=30, loss='warp')
    
    start = time.time()
    # train model 
    print("Fitting model...")
    model_fm_features.fit(plays_sparse.T.tocsr(),user_features = None, item_features = None)    
    stop = time.time()
    total = stop-start
    train_time.append(total)
    recall, ndcg = evaluate(model_fm_feature, test, plays_sparse)
    print("Recall:",recall*100,'%')
    print("Average NDCG:",ndcg*100,'%')
    recall_size.append(recall)
    ndcg_size.append(ndcg)

#plot
df = pd.DataFrame({'n':size,'ndcg':ndcg_size})
sns.pointplot(x="n",y="ndcg",data=df)
plt.title('NDCG by Input Size for Factorization Machine')
plt.xlabel('Number of Users')
plt.ylabel('Time (s)')

### Training/Predict Time

In [ ]:
#plot LightFM training time vs input size
#plot
df = pd.DataFrame({'n':size,'time':train_time})
sns.pointplot(x="n",y="time",data=df)
plt.title('Training Time by Input Size for Factorization Machine')
plt.xlabel('Number of Users')
plt.ylabel('Time (s)')

# 5. Conclusion / Next Steps

# OLD STUFF: 

# Cross Validation and Parameter Tuning (k-fold)

In this section, we use k-fold cross validation to tune hyperparameters and evaluate our models.

### Splitting into test and training sets

In [ ]:
# Cross Validation test
k=5
train_list, test_list, user_count = split_train_test_per_user(plays_sparse,k,20,cross_valid=True)

## Evaluate and tune ALS

Tuning two parameters: number of latent factors and the regularization factor. 

For latent factors, we try values [10,20,30,40,50,60]

For regularization factors, we try [.01,.03,.05,.07]

In [ ]:
start = time.time()
model=implicit.als.AlternatingLeastSquares
ndcg_list,heatmap_list=auto_tune_parameter(4,20,model,plays_sparse,[10,20,30,40,50,60],[.01,.03,.05,.07],param3=None)
stop = time.time()
total = stop-start

In [ ]:
# Plot heatmap of parameter tuning results
sns.set_style("whitegrid")
sns.heatmap(heatmap_list[2], 
            xticklabels=['0.01','0.03','0.05','0.07'], 
            yticklabels=['10','20','30','40','50','60'], 
            cbar_kws={'label':'NDCG Score'})
plt.ylabel("Number of Latent Factors")
plt.xlabel("Regularization Factor")
plt.title("ALS NDCG scores according to Model Parameters")
plt.show()

Here we analyze the scalability of ALS by looking at datasets with 9k, 20k, 60k, and 150k users:

In [ ]:
# The following block imports larger datasets for scaling tests, these expanded CSVs are not available in the repo
files9k = pd.read_csv('lastfm_9000_users.csv', na_filter=False)
files20k = pd.read_csv('lastfm_20k_users.csv', na_filter=False)
files60k = pd.read_csv('lastfm_60k_users.csv', na_filter=False)
files150k = pd.read_csv('lastfm_150k_users.csv', na_filter=False)
files40k = get_users(files150k, 40000)
files = [files9k, files20k, files40k, files60k]

In [ ]:
# Compute the recall and ndcg using optimal parameters for the ALS model on different dataset sizes
size = [9000, 20000, 40000, 60000]
ndcg_size = []
recall_size = []
for i in files:
    model = implicit.als.AlternatingLeastSquares(factors=30, regularization=0.01)

    #create sparse matrix
    plays_sparse = create_sparse_matrix(i).astype('float')
    print('Matrix Sparsity:', calculate_sparsity(plays_sparse))

    train, test, user_count = split_train_test_per_user(plays_sparse, 4, 20)

    # train model 
    print("Fitting model...")
    model.fit(train, show_progress=True)

    recall, ndcg = evaluate(model, test, plays_sparse)
    print("Recall:",recall*100,'%')
    print("Average NDCG:",ndcg*100,'%')
    recall_size.append(recall)
    ndcg_size.append(ndcg)

In [ ]:
# Plot scalability of ALS model
ndcg_size_df = pd.DataFrame({'N':size,
                       'NDCG': ndcg_size})
g = sns.pointplot(x='N', y='NDCG', data=ndcg_size_df)
plt.title('NDCG by Input Size for ALS')
plt.xlabel('Number Of Users')
plt.show()

Here we analyze the catalog coverage of the ALS model:

In [ ]:
# Calculate catalog coverage of ALS model with optimal parameters
model = implicit.als.AlternatingLeastSquares(factors=30, regularization=0.01)
model.fit(plays_sparse)
users = list(df.user_id.unique())
catalog = []
for i in range(0,len(users)):
    for x,y in model.recommend(i,plays_sparse.T.tocsr(), N=20, filter_already_liked_items=True):
        if x not in catalog:
            catalog.append(x)
print('Catalog Coverage is', len(catalog)/plays_sparse.shape[1])

# Tune ALS

Below we will tune the hyperparameters of ALS from a given range of hyperparameters.

In [ ]:
model=implicit.als.AlternatingLeastSquares
ndcg_list,heatmap_list=auto_tune_parameter(4,20,model,plays_sparse,[10,20,30,40,50,60],[.01,.03,.05,.07],param3=None)

sns.set_style("whitegrid")
sns.heatmap(heatmap_list[2], xticklabels=['0.01','0.03','0.05','0.07'], yticklabels=['10','20','30','40','50','60'], cbar_kws={'label':'NDCG Score'})
plt.ylabel("Number of Latent Factors")
plt.xlabel("Regularization Factor")
plt.title("ALS NDCG scores according to Model Parameters")
plt.show()

# Input Size vs Training Time

Below you can run the code using a larger dataset to evaluate performance tiem compared with input tiem

In [ ]:
#helper function to grab needed users
def get_users(df, n):
    sample_userid = df["user_id"].unique()
    sample_userid = np.random.choice(sample_userid, size = n, replace = False)

    #grab rows with sample user id
    df_sample = df[df.user_id.isin(sample_userid)].reset_index(drop = True)

    return df_sample

#in order to run this analysis, you need to download the 150k data at: 
#https://www.dropbox.com/s/qd8rnlxsuq0rjll/last_fm_bigger_data.zip?dl=0

#read in large dataset
df_150 = pd.read_csv('lastfm_150k_users.csv', na_filter=False)
df_150 = df_150.drop(['Unnamed: 0'], axis=1)

# ALS

In [ ]:
#plot ALS training time vs input size
#calculate training time for different input sizes for ALS
import sns
import time

size = [9000,20000,40000,60000,80000]
train_time = list()

#get train data for each input size
for users in size:
    df = get_users(df_150, users)
    plays_sparse = create_sparse_matrix(df)

    # K-Nearest Neighbors
    model = implicit.als.AlternatingLeastSquares(50)
    
    start = time.time()
    # train model 
    print("Fitting model...")
    model.fit(plays_sparse, show_progress=True)
    stop = time.time()
    total = stop-start
    train_time.append(total)

#plot
df = pd.DataFrame({'n':size,'time':train_time})
sns.pointplot(x="n",y="time",data=df)
plt.title('Training Time by Input Size for KNN')
plt.xlabel('Number of Users')
plt.ylabel('Time (s)')